In [1]:
# Chatbot using Hugging Face Transformers
# ----------------------------------------
# This notebook implements a simple conversational chatbot using a small local model.
# It maintains a short-term memory of the conversation and supports a CLI-style flow.

!pip install transformers torch --quiet


In [2]:
# Import dependencies
from transformers import pipeline


In [3]:
# Load model (small model for local use)
def load_model(model_name="distilgpt2"):
    print(f"Loading model: {model_name} ...")
    generator = pipeline(
        "text-generation",
        model=model_name,
        pad_token_id=50256  # EOS token for GPT2 padding
    )
    return generator

generator = load_model()


Loading model: distilgpt2 ...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


In [4]:
# Conversation memory using sliding window
class ChatMemory:
    def __init__(self, max_turns=5):
        self.max_turns = max_turns
        self.history = []

    def update(self, user_input, bot_response):
        self.history.append(f"User: {user_input}\nBot: {bot_response}")
        if len(self.history) > self.max_turns:
            self.history = self.history[-self.max_turns:]

    def get_context(self):
        return "\n".join(self.history)


In [5]:
# Initialize memory
memory = ChatMemory(max_turns=3)


In [6]:
# Define chatbot function (simulating CLI interaction)
def chatbot_loop():
    print("Chatbot ready! Type '/exit' to quit.\n")
    while True:
        user_input = input("User: ")
        if user_input.lower() == "/exit":
            print("Exiting chatbot. Goodbye!")
            break

        prompt = memory.get_context() + f"\nUser: {user_input}\nBot:"
        response = generator(prompt, max_length=200, do_sample=True, temperature=0.7)
        bot_output = response[0]['generated_text'].split("Bot:")[-1].split("User:")[0].strip()

        print(f"Bot: {bot_output}")
        memory.update(user_input, bot_output)


In [7]:
# Run chatbot (works like a CLI inside Jupyter)
chatbot_loop()


Chatbot ready! Type '/exit' to quit.

User: Hii


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Bot: Bii
User: what is Bii?


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Bot: Bii
User: exit


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Bot: the best player
User: /exit
Exiting chatbot. Goodbye!
